In [16]:
# Dependencies and setup

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import requests
import json
import time 
from scipy.stats import linregress

from citipy import citipy
from datetime import date

#Import API key 
weather_api_key = "4343bc3726df924a38d9c0897b39a1a9"

#Output File (CSV)
output_data_file = "output/cities.csv"

# Create range of lats and lons
lat_range = (-90,90)
lng_range = (-180,180)

In [18]:
#List for holding the lat_lon and cities
lat_lng = []
cities = []

#Create random combinations of latitude and longitude 
lats = np.random.uniform(lat_range[0],lat_range[1], size = 1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size = 1500)
lat_lngs = zip(lats, lngs)

#Generate the nearest city for each random combo
for i in lat_lngs:
    city = citipy.nearest_city(i[0], i[1]).city_name

    if city not in cities:
        cities.append(city)

print(len(cities))

595


Make API Calls

In [21]:
#Perform a weather check on each city using a series of successive API calls.
#Include a print log of each city as it's being processed (with the city number and city name).

unit = 'imperial'

url = f"http://api.openweathermap.org/data/2.5/weather?appid={weather_api_key}&units={unit}"

city_data = []

record_count = 1 
set_count = 1 

# Loop through all the cities in our list
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # Create endpoint URL with each city
    city_url = url + "&q=" + city
    
    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    print(city_url)

    # Add 1 to the record count
    record_count += 1

# Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()

        # Parse out the max temp, humidity, and cloudiness
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_date = city_weather["dt"]

        # Append the City information into city_data list
        city_data.append({"City": city, 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

 # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
              
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

=imperial&q=nerchinskiy zavod
Processing Record 18 of Set 10 | kalianget
http://api.openweathermap.org/data/2.5/weather?appid=4343bc3726df924a38d9c0897b39a1a9&units=imperial&q=kalianget
Processing Record 19 of Set 10 | trincomalee
http://api.openweathermap.org/data/2.5/weather?appid=4343bc3726df924a38d9c0897b39a1a9&units=imperial&q=trincomalee
Processing Record 20 of Set 10 | xai-xai
http://api.openweathermap.org/data/2.5/weather?appid=4343bc3726df924a38d9c0897b39a1a9&units=imperial&q=xai-xai
Processing Record 21 of Set 10 | haradok
http://api.openweathermap.org/data/2.5/weather?appid=4343bc3726df924a38d9c0897b39a1a9&units=imperial&q=haradok
Processing Record 22 of Set 10 | alamosa
http://api.openweathermap.org/data/2.5/weather?appid=4343bc3726df924a38d9c0897b39a1a9&units=imperial&q=alamosa
Processing Record 23 of Set 10 | labuhan
http://api.openweathermap.org/data/2.5/weather?appid=4343bc3726df924a38d9c0897b39a1a9&units=imperial&q=labuhan
Processing Record 24 of Set 10 | smithers
http

In [38]:
# Convert array of JSONs into Pandas DataFrame
CityWeather= pd.DataFrame(city_data)

# Show Record Count
CCityWeatherount()

NameError: name 'CCityWeatherount' is not defined

In [39]:
# Display the City Data Frame
CityWeatherdf

NameError: name 'CityWeatherdf' is not defined

Remove cities where humidity > 100%

In [33]:
#Filtering out cities that have humidities that are not feasable.
toohumid = CityWeather.loc[city_data['Humidity'] > 100]
CityWeather=CityWeather.loc[city_data['Humidity'] <= 100]

TypeError: list indices must be integers or slices, not str

In [61]:
#  Get the indices of cities that have humidity over 100%.
print(f'There are {len(toohumid)} entries with humidity over 100%')

There are 0 entries with humidity over 100%


In [62]:
#making sure all varables are equal and no missing data
counts = CityWeather_df.count()
countsinfo = CityWeather_df.describe()
print(counts)
countsinfo

City           0
Country        0
Temperature    0
Humidity       0
Cloudiness     0
Windspeed      0
Date           0
Longitude      0
Latitude       0
dtype: int64


,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Date,Longitude,Latitude
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
# Extract relevant fields from the data frame
scatterplotdata = CityWeather_df[['City', 'Latitude', 'Temperature', 'Humidity', 'Cloudiness', 'Windspeed']]

# Export the City_Data into a csv
scatterplotdata.to_csv(r'/Users/specky3512/GitHub/python-API-challenge/output_data/DataforScatterplots.csv', index = False)

scatterplotdata.head(20)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/specky3512/GitHub/python-API-challenge/output_data/DataforScatterplots.csv'

In [64]:
#creating x & y var
#x var will be able to be used in all charts and will not have to be recreated
x = scatterplotdata['Latitude']
y1 = scatterplotdata['Temperature']
#plotting scatter
plt.scatter(x,y1)
#adding labels & Title
plt.xlabel('Latitude')
plt.ylabel('Temperature(°F)')
plt.title(f'Latitude vs. Temperature {today}')
#adding Grid
plt.grid()
#saving figure
plt.savefig(f'../output_data/Latitude vs. Temperature {today}.png')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '../output_data/Latitude vs. Temperature 2020-09-28.png'

Latitude vs. Humidity

In [65]:
#create var for humidity
y2 = scatterplotdata['Humidity']
#plot scatter graph
plt.scatter(x,y2)
#adding titles/labels
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.title(f'Latitude vs. Humidity {today}')
plt.grid()
#saving figure
plt.savefig(f'../output_data/Latitude vs. Humidity {today}.png')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '../output_data/Latitude vs. Humidity 2020-09-28.png'

In [66]:
#creating var for cloudiness
y3 = scatterplotdata['Cloudiness']
#plotting scatter
plt.scatter(x,y3)
#adding titels/labels
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')
plt.title(f'Latitude vs. Cloudiness {today}')
plt.grid()
#saving figure
plt.savefig(f'../output_data/Latitude vs. Cloudiness {today}.png')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '../output_data/Latitude vs. Cloudiness 2020-09-28.png'

In [67]:
#creating var for windspeed
y4 = scatterplotdata['Windspeed']
#creating scatter plot
plt.scatter(x,y4)
#adding title/labesl
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (MPH)')
plt.title(f'Latitude vs. Wind Speed {today}')
plt.grid()
#saving figure
plt.savefig(f'../output_data/Latitude vs. Wind Speed {today}.png')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '../output_data/Latitude vs. Wind Speed 2020-09-28.png'

Linear Regression

In [68]:
# Create a function to create Linear Regression plots. The function also takes in var for line equation placement on charts
# a couple print statements to print the r-squre value and line equation and adding labels to charts that are going to be the same
def linerarregress(x,y,a,b,c):
    (slope, intercept, rvalue, pvalue, stderr)=linregress(x,y)
    regress_values = x * slope + intercept
    line_eq = 'y=' + str(round(slope,2)) +'x+' + str(round(intercept,2))
    plt.scatter(x,y, color='blue')
    plt.plot(x,regress_values,"r-")
    print(f'The r-square is {rvalue}')
    print(line_eq)
    plt.annotate(line_eq,(a,b),fontsize=15, color='red')
    plt.grid()
    plt.xlabel('Latitude')
    plt.ylabel(f'{c}')

In [69]:
# Create Northern and Southern Hemisphere DataFrames
Northern = scatterplotdata[scatterplotdata['Latitude'] >= 0]

Southern = scatterplotdata[scatterplotdata['Latitude'] < 0]

In [70]:
#Northern Hemisphere - Max Temp vs. Latitude Linear Regression

#setting x var for norther latitude point and setting y var
x_north = Northern["Latitude"]
y_nTemp = Northern["Temperature"]
#making scatter plot
plt.scatter(x_north,y_nTemp)
#adding title
plt.title(f'NorthernLat vs Temp {today}')
#running linear regression function
linerarregress(x_north,y_nTemp,15,40,"Temperature(°F)")
#saving figure
plt.savefig(f'../output_data/NorthernLat vs Temp {today}.png')
plt.show()

ValueError: Inputs must not be empty.

In [71]:
#Southern Hemisphere - Max Temp vs. Latitude Linear Regression

#setting x var for southern latitude point and setting y var
x_south = Southern["Latitude"]
y_sTemp = Southern["Temperature"]
#creating scatter plot
plt.scatter(x_south, y_sTemp)
#adding title
plt.title(f'SouthernLat vs Temp {today}')
#calling linear regression fuction
linerarregress(x_south, y_sTemp,-55,80,"Temperature(°F)")
#saving figure
plt.savefig(f'../output_data/SouthernLat vs Temp {today}.png')
plt.show()

ValueError: Inputs must not be empty.

In [72]:
# Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

#setting y var
y_nHum = Northern["Humidity"]
#creating scatter plot
plt.scatter(x_north, y_nHum)
#adding title
plt.title(f'NorthernLat vs Humidity {today}')
#calling linear regression function
linerarregress(x_north, y_nHum,0,10,'Humidity (%)')
#saving figure
plt.savefig(f'../output_data/NorthernLat vs Humidity {today}.png')
plt.show()

ValueError: Inputs must not be empty.

In [73]:
#Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

#setting y var
y_sHum=Southern["Humidity"]
#creating scatter plot
plt.scatter(x_south,y_sHum)
#adding title
plt.title(f'SouthernLat vs Humidity {today}')
#calling linear regression function
linerarregress(x_south,y_sHum,-50,20,'Humidity (%)')
#saving figure
plt.savefig(f'../output_data/SouthernLat vs Humidity {today}.png')
plt.show()

ValueError: Inputs must not be empty.

In [74]:

#setting y var
y_nClo = Northern["Cloudiness"]
#creating scatter plot
plt.scatter(x_north,y_nClo)
#adding title
plt.title(f'NorthernLat vs Cloudiness {today}')
#calling linear regression function
linerarregress(x_north,y_nClo,5,60,"Cloudiness (%)")
#saving figure
plt.savefig(f'../output_data/NorthernLat vs Cloudiness {today}.png')
plt.show()

ValueError: Inputs must not be empty.

In [75]:
#setting y var
y_sClo = Southern["Cloudiness"]
#creating scatter plot
plt.scatter(x_south,y_sClo)
#adding title
plt.title(f'SouthernLat vs Cloudiness {today}')
#calling linear regression function
linerarregress(x_south,y_sClo,-55,60,"Cloudiness (%)")
#saving figure
plt.savefig(f'../output_data/SouthernLat vs Cloudiness {today}.png')
plt.show()

ValueError: Inputs must not be empty.

In [76]:
#setting y var
y_nWind= Northern["Windspeed"]
#creating scatter plot
plt.scatter(x_north, y_nWind)
#adding title
plt.title(f'NorthernLat vs Wind Speed {today}')
#calling linear regression function
linerarregress(x_north, y_nWind,0,25, "Wind Speed (mph)")
#saving figure
plt.savefig(f'../output_data/NorthernLat vs WindSpeed {today}.png')
plt.show()

ValueError: Inputs must not be empty.

In [77]:
#setting y var
y_sWind = Southern["Windspeed"]
#creating scatter plot
plt.scatter(x_south, y_sWind)
#adding title
plt.title(f'SouthernLat vs Wind Speed {today}')
#calling linear regression function
linerarregress(x_south, y_sWind,-55,15,"Wind Speed (mph)")
#saving figure
plt.savefig(f'../output_data/SouthernLat vs Windspeed {today}.png')
plt.show()

ValueError: Inputs must not be empty.